# Sentinel-2 NDVI z-score cube demo

This notebook streams a Sentinel-2 Level-2A chip with `cubo`, computes NDVI and
z-scores via the CubeDynamics pipe/verb API, and previews the result with
Lexcube.

In [ ]:
from __future__ import annotations
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import cubo
import lexcube
from cubedynamics import pipe, verbs as v

LAT = 43.89  # Kyle, South Dakota
LON = -102.18
START = "2023-06-01"
END = "2024-09-30"
EDGE_SIZE = 512
RESOLUTION = 10
CLOUD_LT = 40

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    s2 = cubo.create(
        lat=LAT,
        lon=LON,
        collection="sentinel-2-l2a",
        bands=["B04", "B08"],
        start_date=START,
        end_date=END,
        edge_size=EDGE_SIZE,
        resolution=RESOLUTION,
        query={"eo:cloud_cover": {"lt": CLOUD_LT}},
    )

if "band" in s2.dims and s2.dims[0] == "band":
    s2 = s2.transpose("time", "y", "x", "band")

ndvi_z = (
    pipe(s2)
    | v.ndvi_from_s2(nir_band="B08", red_band="B04")
    | v.zscore(dim="time", std_eps=1e-4)
).unwrap()

z_med = ndvi_z.median(dim=["y", "x"], skipna=True).to_series()

fig, ax = plt.subplots(figsize=(6, 3))
z_med.plot(ax=ax)
ax.axhline(0.0, ls="--", lw=1, alpha=0.7)
ax.set_ylabel("Median NDVI z-score")
ax.set_xlabel("Date")
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

w = lexcube.Cube3DWidget(
    ndvi_z.clip(-3.0, 3.0),
    cmap="RdBu_r",
    vmin=-3.0,
    vmax=3.0,
    title=f"Sentinel-2 NDVI z-score • {START}→{END}",
)
w.plot()